# 在CIFAR数据集上测试CSE、CSA

In [1]:
import torch 
import numpy as np
import os
import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import sys
import copy
from torch.autograd.gradcheck import zero_gradients
from utils import *


In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = f.relu(self.conv1(x))
        x = f.max_pool2d(x, 2)
        x = f.relu(self.conv2(x))
        x = f.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
# 一般的对抗模型模型
model_ADV = LeNet()
model_ADV.load_state_dict(torch.load('../model/LeNet_CIFAR_adv_e0.03.pt'))

model_NORMAL = LeNet()
model_NORMAL.load_state_dict(torch.load('../model/Lenet_CIFAR.pt'))


<All keys matched successfully>

# clean

In [5]:
NORMALIZE = True

CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
DEVICE = torch.device(cuda_num if torch.cuda.is_available() else "cpu")
batch_size = 64

data_home = 'F:\\work'

if NORMALIZE:
    train_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    test_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
else:
    train_transform = transforms.Compose([transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.ToTensor()])
train_set = torchvision.datasets.CIFAR10(root=os.path.join(data_home, 'dataset/CIFAR10'), train=True, download=True, transform=train_transform)
test_set = torchvision.datasets.CIFAR10(root=os.path.join(data_home, 'dataset/CIFAR10'), train=False, download=True, transform=test_transform)


train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# 记录结果
results_infos = {}

# 先读取未经过对抗训练的模型
# 在进行对抗训练

# 参数
epsilon_model = 0.03
epsilon_attack = 0.03
criterion_CSA = Loss_cost_sensitive()


# 循环 对每一个类分别进行保护
for i_label in range(10):
    ################################
    # 读取模型
    model_CSA = LeNet()
    path_model_CSA = '../model/LeNet_CIFAR_adv_cost_sensitive_'+ str(i_label) +'_e'+ str(epsilon_model) +'.pt'
    model_CSA.load_state_dict(torch.load(path_model_CSA))
    print('load model for initialization: {}'.format(path_model_CSA))
    model_CSA = model_CSA.to(DEVICE)
    
    C = get_cost_matric(i_label)
    C = C.to(DEVICE)
    print('protect label: {}'.format(i_label))
    print('load cost matric: ')
    print(C)
    LABEL = 'Protect Label ' + str(i_label)

    ## 对 model_CSA 的评估
    print('对 model_CSA 的评估')
    results_info = {}
    model_CSA.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model_CSA(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[1] = images_targets
    
    ##################################
    ## 对 model_CSE 的评估
    print('对 model_CSE 的评估')
    # 读取模型
    model_CSE = LeNet()
    path_model_CSE = '../model/LeNet_CIFAR_cost_sensitive_extension_'+str(i_label)+'.pt'
    model_CSE.load_state_dict(torch.load(path_model_CSE))
    print('load model for initialization: {}'.format(path_model_CSE))
    model_CSE = model_CSE.to(DEVICE)
    criterion_CSE = Loss_CSE(model_CSE,1,1,1)

    model_CSE.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model_CSE(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[2] = images_targets
    
    ##################################
    ## 对 model_ADV 的评估
    print('对 model_ADV 的评估')


    model_ADV.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model_ADV(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[3] = images_targets
    
    ##################################
    ## 对 model_NORMAL 的评估
    print('对 model_NORMAL 的评估')

    model_NORMAL.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model_NORMAL(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[4] = images_targets 
    # 记录结果
    results_infos[i_label] = results_info

load model for initialization: ../model/LeNet_CIFAR_adv_cost_sensitive_0_e0.03.pt
protect label: 0
load cost matric: 
tensor([[ 0., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.]])
对 model_CSA 的评估
 1000
 0 correct: 0.678
 1000
 1 correct: 0.78
 1000
 2 correct: 0.491
 1000
 3 correct: 0.366
 1000
 4 correct: 0.596
 1000
 5 correct: 0.555
 1000
 6 correct: 0.763
 1000
 7 correct: 0.702
 1000
 8 correct: 0.811
 1000
 9 correct: 0.657
对 model_CSE 的评估
load model

 8 correct: 0.73
 1000
 9 correct: 0.775
load model for initialization: ../model/LeNet_CIFAR_adv_cost_sensitive_9_e0.03.pt
protect label: 9
load cost matric: 
tensor([[ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0., 10.],
        [10., 10., 10., 10., 10., 10., 10., 10., 10.,  0.]])
对 model_CSA 的评估
 1000
 0 correct: 0.652
 1000
 1 correct: 0.764
 1000
 2 correct: 0.399
 1000
 3 correct: 0.336
 1000
 4 correct: 0.503
 1000
 5 correct: 0.67
 1000
 6 correct: 0.814
 1000
 7 correct: 0.703
 1000
 8 correct: 0.764
 1000
 9 

## 保存

In [8]:
import pandas as pd
output_dirs = './output/cifar'
if os.path.exists(output_dirs) is False:
    os.makedirs(output_dirs)

I_avg = {'CSA':[], 'CSE':[], 'ADV':[], 'NORMAL':[]}
writer = pd.ExcelWriter(os.path.join(output_dirs, 'CIFAR_clean_0.03.xlsx'))
for i in results_infos.keys():
    tmp = results_infos[i]
    
    I_avg['CSA'].append(tmp[1][i][1])
    I_avg['CSE'].append(tmp[2][i][1])
    I_avg['ADV'].append(tmp[3][i][1])
    I_avg['NORMAL'].append(tmp[4][i][1])
    
    df  = pd.DataFrame(tmp)
    df.columns = ['CSA', 'CSE', 'ADV', 'NORMAL']
    df = pd.DataFrame([df[i].apply(lambda x: x[1]) for i in df.columns])
    df = df.sort_index()
    
    df.to_excel(writer, sheet_name=str(i))
writer.save()

print('I of CSA: {}'.format(np.mean(I_avg['CSA'])))
print('I of CSE: {}'.format(np.mean(I_avg['CSE'])))
print('I of ADV: {}'.format(np.mean(I_avg['ADV'])))
print('I of NORMAL: {}'.format(np.mean(I_avg['NORMAL'])))

I of CSA: 0.6057999999999999
I of CSE: 0.5609999999999999
I of ADV: 0.6444
I of NORMAL: 0.6549000000000001


In [9]:
I_avg

{'CSA': [0.678, 0.746, 0.365, 0.412, 0.475, 0.481, 0.791, 0.628, 0.794, 0.688],
 'CSE': [0.568, 0.741, 0.397, 0.235, 0.506, 0.324, 0.744, 0.63, 0.743, 0.722],
 'ADV': [0.697, 0.732, 0.533, 0.439, 0.595, 0.532, 0.773, 0.669, 0.748, 0.726],
 'NORMAL': [0.74,
  0.759,
  0.494,
  0.495,
  0.583,
  0.536,
  0.699,
  0.738,
  0.73,
  0.775]}

# FGSM

In [11]:
NORMALIZE = True

CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
DEVICE = torch.device(cuda_num if torch.cuda.is_available() else "cpu")
batch_size = 64

data_home = 'F:\\work'

if NORMALIZE:
    train_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    test_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
else:
    train_transform = transforms.Compose([transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.ToTensor()])
train_set = torchvision.datasets.CIFAR10(root=os.path.join(data_home, 'dataset/CIFAR10'), train=True, download=True, transform=train_transform)
test_set = torchvision.datasets.CIFAR10(root=os.path.join(data_home, 'dataset/CIFAR10'), train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

criterion = nn.CrossEntropyLoss()

Files already downloaded and verified
Files already downloaded and verified


In [12]:
# 记录结果
results_infos = {}

# 先读取未经过对抗训练的模型
# 在进行对抗训练

# 参数
epsilon_model = 0.03
epsilon_attack = 0.03
criterion_CSA = Loss_cost_sensitive()


# 循环 对每一个类分别进行保护
for i_label in range(10):
    ################################
    # 读取模型
    model_CSA = LeNet()
    path_model_CSA = '../model/LeNet_CIFAR_adv_cost_sensitive_'+ str(i_label) +'_e'+ str(epsilon_model) +'.pt'
    model_CSA.load_state_dict(torch.load(path_model_CSA))
    print('load model for initialization: {}'.format(path_model_CSA))
    model_CSA = model_CSA.to(DEVICE)
    
    
    C = get_cost_matric(i_label)
    C = C.to(DEVICE)
    print('protect label: {}'.format(i_label))
    print('load cost matric: ')
    print(C)
    LABEL = 'Protect Label ' + str(i_label)

    ## 对 model_CSA 的评估
    print('对 model_CSA 的评估')
    results_info = {}
    model_CSA.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data, sign = my_fgsm(data, target, model_CSA, criterion_CSA, epsilon_attack, DEVICE, C)
            output = model_CSA(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[1] = images_targets
    
##################################
    ## 对 model_CSE 的评估
    print('对 model_CSE 的评估')
    # 读取模型
    model_CSE = LeNet()
    path_model_CSE = '../model/LeNet_CIFAR_cost_sensitive_extension_'+str(i_label)+'.pt'
    model_CSE.load_state_dict(torch.load(path_model_CSE))
    print('load model for initialization: {}'.format(path_model_CSE))
    model_CSE = model_CSE.to(DEVICE)
    criterion_CSE = Loss_CSE(model_CSE,1,1,1)

    model_CSE.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data, sign = my_fgsm(data, target, model_CSE, criterion_CSE, epsilon_attack, DEVICE, C)
            output = model_CSE(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[2] = images_targets
    
##################################
    ## 对 model_ADV 的评估
    print('对 model_ADV 的评估')


    model_ADV.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data, sign = my_fgsm(data, target, model_ADV, criterion, epsilon_attack, DEVICE)
            output = model_ADV(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[3] = images_targets
    
##################################
    ## 对 model_NORMAL 的评估
    print('对 model_NORMAL 的评估')

    model_NORMAL.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data, sign = my_fgsm(data, target, model_NORMAL, criterion, epsilon_attack, DEVICE)
            output = model_NORMAL(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[4] = images_targets 
    # 记录结果
    results_infos[i_label] = results_info

load model for initialization: ../model/LeNet_CIFAR_adv_cost_sensitive_0_e0.03.pt
protect label: 0
load cost matric: 
tensor([[ 0., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.]])
对 model_CSA 的评估
 1000
 0 correct: 0.523
 1000
 1 correct: 0.619
 1000
 2 correct: 0.306
 1000
 3 correct: 0.167
 1000
 4 correct: 0.332
 1000
 5 correct: 0.348
 1000
 6 correct: 0.571
 1000
 7 correct: 0.562
 1000
 8 correct: 0.649
 1000
 9 correct: 0.478
对 model_CSE 的评估
load mode

 7 correct: 0.395
 1000
 8 correct: 0.327
 1000
 9 correct: 0.405
load model for initialization: ../model/LeNet_CIFAR_adv_cost_sensitive_9_e0.03.pt
protect label: 9
load cost matric: 
tensor([[ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0., 10.],
        [10., 10., 10., 10., 10., 10., 10., 10., 10.,  0.]])
对 model_CSA 的评估
 1000
 0 correct: 0.484
 1000
 1 correct: 0.589
 1000
 2 correct: 0.224
 1000
 3 correct: 0.17
 1000
 4 correct: 0.269
 1000
 5 correct: 0.507
 1000
 6 correct: 0.626
 1000
 7 correct: 0.543
 1000
 8

In [13]:
import pandas as pd
output_dirs = './output/cifar'
if os.path.exists(output_dirs) is False:
    os.makedirs(output_dirs)

I_avg = {'CSA':[], 'CSE':[], 'ADV':[], 'NORMAL':[]}
writer = pd.ExcelWriter(os.path.join(output_dirs, 'CIFAR_FGSM_0.03.xlsx'))
for i in results_infos.keys():
    tmp = results_infos[i]
    
    I_avg['CSA'].append(tmp[1][i][1])
    I_avg['CSE'].append(tmp[2][i][1])
    I_avg['ADV'].append(tmp[3][i][1])
    I_avg['NORMAL'].append(tmp[4][i][1])
    
    df  = pd.DataFrame(tmp)
    df.columns = ['CSA', 'CSE', 'ADV', 'NORMAL']
    df = pd.DataFrame([df[i].apply(lambda x: x[1]) for i in df.columns])
    df = df.sort_index()
    
    df.to_excel(writer, sheet_name=str(i))
writer.save()

print('I of CSA: {}'.format(np.mean(I_avg['CSA'])))
print('I of CSE: {}'.format(np.mean(I_avg['CSE'])))
print('I of ADV: {}'.format(np.mean(I_avg['ADV'])))
print('I of NORMAL: {}'.format(np.mean(I_avg['NORMAL'])))

I of CSA: 0.43839999999999996
I of CSE: 0.3115
I of ADV: 0.4407
I of NORMAL: 0.2806


# deep fool

In [4]:
def deepfool(image, net, num_classes=10, overshoot=0.02, max_iter=3):

    """
       :param image: Image of size HxWx3
       :param net: network (input: images, output: values of activation **BEFORE** softmax).
       :param num_classes: num_classes (limits the number of classes to test against, by default = 10)
       :param overshoot: used as a termination criterion to prevent vanishing updates (default = 0.02).
       :param max_iter: maximum number of iterations for deepfool (default = 50)
       :return: minimal perturbation that fools the classifier, number of iterations that it required, new estimated_label and perturbed image
    """
    is_cuda = torch.cuda.is_available()

    if is_cuda:
#         print("Using GPU")
        image = image.cuda()
        net = net.cuda()
    else:
        pass
#         print("Using CPU")


    f_image = net.forward(Variable(image[None, :, :, :], requires_grad=True)).data.cpu().numpy().flatten()
    I = (np.array(f_image)).flatten().argsort()[::-1]

    I = I[0:num_classes]
    label = I[0]

    input_shape = image.cpu().numpy().shape
    pert_image = copy.deepcopy(image)
    w = np.zeros(input_shape)
    r_tot = np.zeros(input_shape)

    loop_i = 0

    x = Variable(pert_image[None, :], requires_grad=True)
    fs = net.forward(x)
    fs_list = [fs[0,I[k]] for k in range(num_classes)]
    k_i = label

    while k_i == label and loop_i < max_iter:

        pert = np.inf
        fs[0, I[0]].backward(retain_graph=True)
        grad_orig = x.grad.data.cpu().numpy().copy()

        for k in range(1, num_classes):
            zero_gradients(x)

            fs[0, I[k]].backward(retain_graph=True)
            cur_grad = x.grad.data.cpu().numpy().copy()

            # set new w_k and new f_k
            w_k = cur_grad - grad_orig
            f_k = (fs[0, I[k]] - fs[0, I[0]]).data.cpu().numpy()

            pert_k = abs(f_k)/np.linalg.norm(w_k.flatten())

            # determine which w_k to use
            if pert_k < pert:
                pert = pert_k
                w = w_k

        # compute r_i and r_tot
        # Added 1e-4 for numerical stability
        r_i =  (pert+1e-4) * w / np.linalg.norm(w)
        r_tot = np.float32(r_tot + r_i)

        if is_cuda:
            pert_image = image + (1+overshoot)*torch.from_numpy(r_tot).cuda()
        else:
            pert_image = image + (1+overshoot)*torch.from_numpy(r_tot)

        x = Variable(pert_image, requires_grad=True)
        fs = net.forward(x)
        k_i = np.argmax(fs.data.cpu().numpy().flatten())

        loop_i += 1

    r_tot = (1+overshoot)*r_tot

    return r_tot, loop_i, label, k_i, pert_image



In [5]:
NORMALIZE = True

CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
DEVICE = torch.device(cuda_num if torch.cuda.is_available() else "cpu")
batch_size = 1

data_home = 'F:\\work'

if NORMALIZE:
    train_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    test_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
else:
    train_transform = transforms.Compose([transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.ToTensor()])
train_set = torchvision.datasets.CIFAR10(root=os.path.join(data_home, 'dataset/CIFAR10'), train=True, download=True, transform=train_transform)
test_set = torchvision.datasets.CIFAR10(root=os.path.join(data_home, 'dataset/CIFAR10'), train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

criterion = nn.CrossEntropyLoss()

Files already downloaded and verified
Files already downloaded and verified


In [6]:
# 记录结果
results_infos = {}

# 先读取未经过对抗训练的模型
# 在进行对抗训练

# 参数
epsilon_model = 0.03
epsilon_attack = 0.03
criterion_CSA = Loss_cost_sensitive()


# 循环 对每一个类分别进行保护
for i_label in range(10):
    ################################
    # 读取模型
    model_CSA = LeNet()
    path_model_CSA = '../model/LeNet_CIFAR_adv_cost_sensitive_'+ str(i_label) +'_e'+ str(epsilon_model) +'.pt'
    model_CSA.load_state_dict(torch.load(path_model_CSA))
    print('load model for initialization: {}'.format(path_model_CSA))
    model_CSA = model_CSA.to(DEVICE)
    
    
    C = get_cost_matric(i_label)
    C = C.to(DEVICE)
    print('protect label: {}'.format(i_label))
    print('load cost matric: ')
    print(C)
    LABEL = 'Protect Label ' + str(i_label)

    ## 对 model_CSA 的评估
    print('对 model_CSA 的评估')
    results_info = {}
    model_CSA.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = data.reshape(3,32,32)
            r, loop_i, label_orig, label_pert, pert_image = deepfool(data, model_CSA)
            output = model_CSA(pert_image)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += 1
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[1] = images_targets
    
##################################
    ## 对 model_CSE 的评估
    print('对 model_CSE 的评估')
    # 读取模型
    model_CSE = LeNet()
    path_model_CSE = '../model/LeNet_CIFAR_cost_sensitive_extension_'+str(i_label)+'.pt'
    model_CSE.load_state_dict(torch.load(path_model_CSE))
    print('load model for initialization: {}'.format(path_model_CSE))
    model_CSE = model_CSE.to(DEVICE)
    criterion_CSE = Loss_CSE(model_CSE,1,1,1)

    model_CSE.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = data.reshape(3,32,32)
            r, loop_i, label_orig, label_pert, pert_image = deepfool(data, model_CSE)
            output = model_CSE(pert_image)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += 1
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[2] = images_targets
    
##################################
    ## 对 model_ADV 的评估
    print('对 model_ADV 的评估')


    model_ADV.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = data.reshape(3,32,32)
            r, loop_i, label_orig, label_pert, pert_image = deepfool(data, model_ADV)
            output = model_ADV(pert_image)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += 1
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[3] = images_targets
    
##################################
    ## 对 model_NORMAL 的评估
    print('对 model_NORMAL 的评估')

    model_NORMAL.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = data.reshape(3,32,32)
            r, loop_i, label_orig, label_pert, pert_image = deepfool(data, model_NORMAL)
            output = model_NORMAL(pert_image)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            count += 1
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[4] = images_targets 
    # 记录结果
    results_infos[i_label] = results_info

load model for initialization: ../model/LeNet_CIFAR_adv_cost_sensitive_0_e0.03.pt
protect label: 0
load cost matric: 
tensor([[ 0., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.]])
对 model_CSA 的评估
 1000
 0 correct: 0.305
 1000
 1 correct: 0.238
 1000
 2 correct: 0.251
 1000
 3 correct: 0.264
 1000
 4 correct: 0.238
 1000
 5 correct: 0.253
 1000
 6 correct: 0.262
 1000
 7 correct: 0.348
 1000
 8 correct: 0.243
 1000
 9 correct: 0.252
对 model_CSE 的评估
load mode

 1000
 8 correct: 0.206
 1000
 9 correct: 0.165
load model for initialization: ../model/LeNet_CIFAR_adv_cost_sensitive_9_e0.03.pt
protect label: 9
load cost matric: 
tensor([[ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0., 10.],
        [10., 10., 10., 10., 10., 10., 10., 10., 10.,  0.]])
对 model_CSA 的评估
 1000
 0 correct: 0.31
 1000
 1 correct: 0.227
 1000
 2 correct: 0.226
 1000
 3 correct: 0.301
 1000
 4 correct: 0.253
 1000
 5 correct: 0.25
 1000
 6 correct: 0.255
 1000
 7 correct: 0.334
 1000
 8 correct: 0.228
 10

In [7]:
import pandas as pd
output_dirs = './output/cifar'
if os.path.exists(output_dirs) is False:
    os.makedirs(output_dirs)

I_avg = {'CSA':[], 'CSE':[], 'ADV':[], 'NORMAL':[]}
writer = pd.ExcelWriter(os.path.join(output_dirs, 'CIFAR_deepfool_0.03.xlsx'))
for i in results_infos.keys():
    tmp = results_infos[i]
    
    I_avg['CSA'].append(tmp[1][i][1])
    I_avg['CSE'].append(tmp[2][i][1])
    I_avg['ADV'].append(tmp[3][i][1])
    I_avg['NORMAL'].append(tmp[4][i][1])
    
    df  = pd.DataFrame(tmp)
    df.columns = ['CSA', 'CSE', 'ADV', 'NORMAL']
    df = pd.DataFrame([df[i].apply(lambda x: x[1]) for i in df.columns])
    df = df.sort_index()
    
    df.to_excel(writer, sheet_name=str(i))
writer.save()

print('I of CSA: {}'.format(np.mean(I_avg['CSA'])))
print('I of CSE: {}'.format(np.mean(I_avg['CSE'])))
print('I of ADV: {}'.format(np.mean(I_avg['ADV'])))
print('I of NORMAL: {}'.format(np.mean(I_avg['NORMAL'])))

I of CSA: 0.23279999999999998
I of CSE: 0.2796
I of ADV: 0.26330000000000003
I of NORMAL: 0.22210000000000002


# PGD

In [8]:
def pgd_attack(model, images, labels, eps=0.3, alpha=2/255, iters=40, loss=None, device=None, C=None):
    if loss==None:
        loss = nn.CrossEntropyLoss()
        
    ori_images = images.data
    
    for i in range(iters) :  
        images.requires_grad = True
        outputs = model(images)

        model.zero_grad()
        if C==None:
            cost = loss(outputs, labels).to(device)
        else:
            cost = loss(outputs, labels, C)
        cost.backward()
        adv_images = images + alpha*images.grad.sign()
        eta = torch.clamp(adv_images - ori_images, min=-eps, max=eps)
#         images = torch.clamp(ori_images + eta, min=0, max=1).detach_()
        images = (ori_images + eta).detach_()
        images = torch.clamp(ori_images + eta, min=-0.4242, max=2.8214).detach_()
    
    return images

In [9]:
NORMALIZE = True

CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
DEVICE = torch.device(cuda_num if torch.cuda.is_available() else "cpu")
batch_size = 64

data_home = 'F:\\work'

if NORMALIZE:
    train_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    test_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
else:
    train_transform = transforms.Compose([transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.ToTensor()])
train_set = torchvision.datasets.CIFAR10(root=os.path.join(data_home, 'dataset/CIFAR10'), train=True, download=True, transform=train_transform)
test_set = torchvision.datasets.CIFAR10(root=os.path.join(data_home, 'dataset/CIFAR10'), train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
# 记录结果
results_infos = {}

# 先读取未经过对抗训练的模型
# 在进行对抗训练

# 参数
epsilon_model = 0.3
epsilon_attack = 0.3
criterion_CSA = Loss_cost_sensitive()


# 循环 对每一个类分别进行保护
for i_label in range(10):
    ################################
    # 读取模型
    model_CSA = LeNet()
    path_model_CSA = '../model/LeNet_CIFAR_adv_cost_sensitive_'+ str(i_label) +'_e'+ str(epsilon_model) +'.pt'
    model_CSA.load_state_dict(torch.load(path_model_CSA))
    print('load model for initialization: {}'.format(path_model_CSA))
    model_CSA = model_CSA.to(DEVICE)
    
    
    C = get_cost_matric(i_label)
    C = C.to(DEVICE)
    print('protect label: {}'.format(i_label))
    print('load cost matric: ')
    print(C)
    LABEL = 'Protect Label ' + str(i_label)

    ## 对 model_CSA 的评估
    print('对 model_CSA 的评估')
    results_info = {}
    model_CSA.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = pgd_attack(model_CSA, data, target, eps=0.3, alpha=0.3, iters=40, loss=criterion_CSA, C=C)
            output = model_CSA(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[1] = images_targets
    
##################################
    ## 对 model_CSE 的评估
    print('对 model_CSE 的评估')
    # 读取模型
    model_CSE = LeNet()
    path_model_CSE = '../model/LeNet_CIFAR_cost_sensitive_extension_'+str(i_label)+'.pt'
    model_CSE.load_state_dict(torch.load(path_model_CSE))
    print('load model for initialization: {}'.format(path_model_CSE))
    model_CSE = model_CSE.to(DEVICE)
    criterion_CSE = Loss_CSE(model_CSE,1,1,1)

    model_CSE.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = pgd_attack(model_CSE, data, target, eps=0.3, alpha=0.3, iters=40, loss=criterion_CSE, C=C)
            output = model_CSE(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[2] = images_targets
    
##################################
    ## 对 model_ADV 的评估
    print('对 model_ADV 的评估')


    model_ADV.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = pgd_attack(model_ADV, data, target, eps=0.3, alpha=0.3, iters=40, loss=criterion)
            output = model_ADV(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[3] = images_targets
    
##################################
    ## 对 model_NORMAL 的评估
    print('对 model_NORMAL 的评估')

    model_NORMAL.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = pgd_attack(model_NORMAL, data, target, eps=0.3, alpha=0.3, iters=40, loss=criterion)
            output = model_NORMAL(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[4] = images_targets 
    # 记录结果
    results_infos[i_label] = results_info

load model for initialization: ../model/LeNet_CIFAR_adv_cost_sensitive_0_e0.3.pt
protect label: 0
load cost matric: 
tensor([[ 0., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.]])
对 model_CSA 的评估
 1000
 0 correct: 0.0
 1000
 1 correct: 0.0
 1000
 2 correct: 0.009
 1000
 3 correct: 0.006
 1000
 4 correct: 0.011
 1000
 5 correct: 0.001
 1000
 6 correct: 0.007
 1000
 7 correct: 0.001
 1000
 8 correct: 0.004
 1000
 9 correct: 0.001
对 model_CSE 的评估
load model for

        [10., 10., 10., 10., 10., 10., 10., 10., 10.,  0.]])
对 model_CSA 的评估
 1000
 0 correct: 0.002
 1000
 1 correct: 0.0
 1000
 2 correct: 0.007
 1000
 3 correct: 0.008
 1000
 4 correct: 0.009
 1000
 5 correct: 0.004
 1000
 6 correct: 0.005
 1000
 7 correct: 0.0
 1000
 8 correct: 0.001
 1000
 9 correct: 0.0
对 model_CSE 的评估
load model for initialization: ../model/LeNet_CIFAR_cost_sensitive_extension_9.pt
 1000
 0 correct: 0.001
 1000
 1 correct: 0.0
 1000
 2 correct: 0.003
 1000
 3 correct: 0.0
 1000
 4 correct: 0.016
 1000
 5 correct: 0.0
 1000
 6 correct: 0.0
 1000
 7 correct: 0.0
 1000
 8 correct: 0.001
 1000
 9 correct: 0.0
对 model_ADV 的评估
 1000
 0 correct: 0.012
 1000
 1 correct: 0.0
 1000
 2 correct: 0.026
 1000
 3 correct: 0.0
 1000
 4 correct: 0.03
 1000
 5 correct: 0.002
 1000
 6 correct: 0.001
 1000
 7 correct: 0.004
 1000
 8 correct: 0.0
 1000
 9 correct: 0.0
对 model_NORMAL 的评估
 1000
 0 correct: 0.0
 1000
 1 correct: 0.0
 1000
 2 correct: 0.005
 1000
 3 correct: 0.0
 1000
 

In [11]:
import pandas as pd
output_dirs = './output/cifar'
if os.path.exists(output_dirs) is False:
    os.makedirs(output_dirs)

I_avg = {'CSA':[], 'CSE':[], 'ADV':[], 'NORMAL':[]}
writer = pd.ExcelWriter(os.path.join(output_dirs, 'CIFAR_PDG_0.03.xlsx'))
for i in results_infos.keys():
    tmp = results_infos[i]
    
    I_avg['CSA'].append(tmp[1][i][1])
    I_avg['CSE'].append(tmp[2][i][1])
    I_avg['ADV'].append(tmp[3][i][1])
    I_avg['NORMAL'].append(tmp[4][i][1])
    
    df  = pd.DataFrame(tmp)
    df.columns = ['CSA', 'CSE', 'ADV', 'NORMAL']
    df = pd.DataFrame([df[i].apply(lambda x: x[1]) for i in df.columns])
    df = df.sort_index()
    
    df.to_excel(writer, sheet_name=str(i))
writer.save()

print('I of CSA: {}'.format(np.mean(I_avg['CSA'])))
print('I of CSE: {}'.format(np.mean(I_avg['CSE'])))
print('I of ADV: {}'.format(np.mean(I_avg['ADV'])))
print('I of NORMAL: {}'.format(np.mean(I_avg['NORMAL'])))

I of CSA: 0.0036
I of CSE: 0.0015
I of ADV: 0.0078
I of NORMAL: 0.001
